# Ray Tune generic black-box optimiser

[Ray Tune](https://docs.ray.io/en/latest/tune/index.html) is a hyperparameter tuning library primarily developed for machine learning. However, it is suitable for generic black-box optimisation as well. For our purpose, it provides an interface for running simulations inside a given *search space* and optimising for a given *loss function* $\mathcal{L}$ using a given *algorithm*. It automatically manages checkpointing, logging and, importantly, parallel (or even distributed) computing.

Details for installing Ray can be found [here](https://docs.ray.io/en/latest/ray-overview/installation.html), but the regular pip install should work for most. Notably, ARM-based macOS support is experimental. 

```console
pip install -U "ray[tune,air]"
```

In this notebook, we implement a toy example of optimising a `mmi1x2` component for a transmission of $|S_{21}|^2 = 0.3$ for a given wavelength using Meep.

In [1]:
import re
import json
import shutil
import subprocess
import importlib
import tempfile
from pathlib import Path
from functools import partial

import numpy as np
import pandas as pd
import ray
import ray.air
import ray.air.session
from ray import tune

import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm
from gdsfactory.config import PATH

tmp = PATH.optimiser
tmp.mkdir(exist_ok=True)

2023-01-12 18:26:37.850 | INFO     | gdsfactory.config:<module>:48 - Load '/workspaces/gdsfactory/gdsfactory' 6.18.3


Using MPI version 4.0, 1 processes


2023-01-12 18:26:41.610 | INFO     | gdsfactory.simulation.gmeep:<module>:34 - Meep '1.25.0-beta' installed at ['/opt/conda/lib/python3.8/site-packages/meep']


## Loss function $\mathcal{L}$

The loss function is very important and should be designed to be meaningful for your need.

The easiest method to optimise for a specific value is to use $L_1$ or $L_2$ (MSE) loss. Different optimisation algorithms might prefer more or less aggressive behaviour close to target, so choose depending on that.
$$
\begin{align*}
    L_1(x) &= |x_\text{target} - x|, \\
    L_2(x) &= \left(x_\text{target} - x\right)^2
    .
\end{align*}
$$

In [2]:
def loss_S21_L1(x, target):
    r"""Loss function. Returns :math:`$\sum_i L_1(x_i)$` and :math:`$x$` as a tuple"""
    return np.abs(target - x), x

Let's select a target of $0.7$ for $S_{21}$

In [3]:
loss = partial(loss_S21_L1, target=0.3)

## Optimisation settings

Here we specify the search space, the optimiser and its settings.

<div class="alert alert-block alert-info"> 
    <b>Note</b> Choosing a new optimiser often requires you to install a separate package, see <a href="https://docs.ray.io/en/latest/tune/api_docs/suggestion.html">Ray Tune → Search Algoritms</a> for details. Here one needs to install <a href="http://hyperopt.github.io/hyperopt/">Hyperopt</a>.
</div>

In [4]:
search_config = {
    'length_mmi': tune.uniform(0.05, 2),
    'width_mmi': tune.uniform(0.05, 2),
}

# pylint: disable=wrong-import-position,ungrouped-imports
from ray.tune.search.hyperopt import HyperOptSearch

tune_config = tune.TuneConfig(
    metric='loss',
    mode='min',
    search_alg=HyperOptSearch(),
    max_concurrent_trials=2,  # simulations to run in parallel
    num_samples=-1,  # max iterations, can be -1 for infinite
    time_budget_s=60*20,  # time after which optimisation is stopped. May be useful along with ``num_samples=-1``.
)

## Implement a *trainable* function

We need to implement a function which can be *trained* to be improved w.r.t. our $\mathcal{L}$. In other words, we create a function for a single training step, which generates, runs, and returns output $\mathcal{L}(\vec{x})$ from simulations for given parameters $\vec{x}$. This may require a bit more effort and some shell scripting to get right depending on your simulations.

Here we demonstrate a trainable for S-parameter simulations. The `write_sparameters_meep` returns $\mathbf{S}$ as a function of $\lambda$ given in $\text{µm}$. From this, we select $S_{21}(\lambda)$ and try to optimise for $\min_\text{geometry} \sum_\lambda (S_{21}(\lambda) - \text{target})$. In other words, that the transmission from 1 to 2 would be as close to target as possible for the given wavelength (or range of wavelengths).

In [5]:
def trainable_simulations(config):
    """Training step, or `trainable`, function for Ray Tune to run simulations and return results."""

    # Component to optimise
    component = gf.components.mmi1x2(**config)
    
    # Simulate and get output
    dirpath = tmp / ray.air.session.get_trial_id()

    meep_params = dict(
        component=component,
        run=True,
        dirpath=dirpath,
        wavelength_start=1.5,
        # wavelength_stop=1.6,
        wavelength_points=1,
    )

    if (use_mpi := True):  # change this to false if no MPI support
        s_params = gm.write_sparameters_meep_mpi(
            cores=2,  # set this to be same as in `tune.Tuner`
            **meep_params
        )
        s_params = np.load(s_params) # parallel version returns filepath to npz instead
    else:
        s_params = gm.write_sparameters_meep(**meep_params)

    s_params_abs = np.abs(s_params['o2@0,o1@0']) ** 2

    loss_x, x = loss(s_params_abs)
    if not np.isscalar(x):  # for many wavelengths, consider sum and mean
        loss_x, x = loss_x.sum(), x.mean()

    return {'loss': loss_x, 'value': x}

    # ALTERNATIVE
    # For a shell-based solution to more software, subprocess.run is recommended roughly as below
    # interpreter = shutil.which('bash')
    # subprocess.run(
    #     [interpreter, '-c', './generated_simulation.sh'],
    #     cwd=dirpath,
    #     check=True,
    # )

## Run optimiser
In the cell below, we gather all the code above to a [`tune.Tuner`](https://docs.ray.io/en/latest/tune/api_docs/execution.html#tuner) instance and start the optimisation by calling `tuner.fit()`.

In [6]:
tuner = tune.Tuner(
    tune.with_resources(trainable_simulations, {"cpu": 2}),  # maximum resources given to a worker, 'gpu' is supported
    param_space=search_config,
    tune_config=tune_config,
    run_config=ray.air.RunConfig(
        local_dir=tmp / "ray_results",
        checkpoint_config=ray.air.CheckpointConfig(checkpoint_frequency=1),
        log_to_file=True,
        verbose=2,  # Intermediate results in Jupyter
    )
)

# Previous simulations can be restored with, see https://docs.ray.io/en/latest/tune/tutorials/tune-stopping.html
# tuner = Tuner.restore(path=tmp / "ray_results/my_experiment")

results = tuner.fit()

2023-01-12 18:26:43,860	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(pid=20689) 2023-01-12 18:26:46.647 | INFO     | gdsfactory.config:<module>:48 - Load '/workspaces/gdsfactory/gdsfactory' 6.18.3


(pid=20689) Using MPI version 4.0, 1 processes


(pid=20689) 2023-01-12 18:26:50.509 | INFO     | gdsfactory.simulation.gmeep:<module>:34 - Meep '1.25.0-beta' installed at ['/opt/conda/lib/python3.8/site-packages/meep']
(trainable_simulations pid=20689) 2023-01-12 18:26:50.572 | INFO     | gdsfactory.component:_write_library:1494 - Wrote to '/workspaces/gdsfactory/gdslib/sp/temp/write_sparameters_meep_mpi.gds'
(trainable_simulations pid=20689) 2023-01-12 18:26:50.587 | INFO     | gdsfactory.component:write_gds_with_metadata:1579 - Write YAML metadata to '/workspaces/gdsfactory/gdslib/sp/temp/write_sparameters_meep_mpi.yml'
(trainable_simulations pid=20689) 2023-01-12 18:26:50.591 | INFO     | gdsfactory.simulation.gmeep.write_sparameters_meep_mpi:write_sparameters_meep_mpi:197 - mpirun -np 2 /opt/conda/bin/python /workspaces/gdsfactory/gdslib/sp/temp/write_sparameters_meep_mpi.py
(trainable_simulations pid=20689) 2023-01-12 18:26:50.591 | INFO     | gdsfactory.simulation.gmeep.write_sparameters_meep_mpi:write_sparameters_meep_mpi:198

(pid=20796) Using MPI version 4.0, 1 processes


...

(pid=28552) Using MPI version 4.0, 1 processes


(pid=28552) 2023-01-12 18:46:50.671 | INFO     | gdsfactory.simulation.gmeep:<module>:34 - Meep '1.25.0-beta' installed at ['/opt/conda/lib/python3.8/site-packages/meep']
(trainable_simulations pid=28552) 2023-01-12 18:46:50.757 | INFO     | gdsfactory.component:_write_library:1494 - Wrote to '/workspaces/gdsfactory/gdslib/sp/temp/write_sparameters_meep_mpi.gds'
(trainable_simulations pid=28552) 2023-01-12 18:46:50.783 | INFO     | gdsfactory.component:write_gds_with_metadata:1579 - Write YAML metadata to '/workspaces/gdsfactory/gdslib/sp/temp/write_sparameters_meep_mpi.yml'
(trainable_simulations pid=28552) 2023-01-12 18:46:50.792 | INFO     | gdsfactory.simulation.gmeep.write_sparameters_meep_mpi:write_sparameters_meep_mpi:197 - mpirun -np 2 /opt/conda/bin/python /workspaces/gdsfactory/gdslib/sp/temp/write_sparameters_meep_mpi.py
(trainable_simulations pid=28552) 2023-01-12 18:46:50.792 | INFO     | gdsfactory.simulation.gmeep.write_sparameters_meep_mpi:write_sparameters_meep_mpi:198

(trainable_simulations pid=28380) Using MPI version 4.0, 2 processes
(trainable_simulations pid=28380) [18:46:47] INFO     Using client version: 1.8.1                  __init__.py:112
(trainable_simulations pid=28380) 
(trainable_simulations pid=28380) Elapsed run time = 65.6374 s
(trainable_simulations pid=28380) 
(trainable_simulations pid=28380) 2023-01-12 18:46:41.974 | INFO     | gdsfactory.config:<module>:48 - Load '/workspaces/gdsfactory/gdsfactory' 6.18.3
(trainable_simulations pid=28380) 2023-01-12 18:46:42.004 | INFO     | gdsfactory.config:<module>:48 - Load '/workspaces/gdsfactory/gdsfactory' 6.18.3
(trainable_simulations pid=28380) 2023-01-12 18:46:46.338 | INFO     | gdsfactory.simulation.gmeep:<module>:34 - Meep '1.25.0-beta' installed at ['/opt/conda/lib/python3.8/site-packages/meep']
(trainable_simulations pid=28380) 2023-01-12 18:46:46.339 | INFO     | gdsfactory.simulation.gmeep:<module>:34 - Meep '1.25.0-beta' installed at ['/opt/conda/lib/python3.8/site-packages/me

2023-01-12 18:47:59,594	INFO tune.py:762 -- Total run time: 1274.56 seconds (1205.59 seconds for the tuning loop).


(trainable_simulations pid=28552) Using MPI version 4.0, 2 processes
(trainable_simulations pid=28552) [18:46:57] INFO     Using client version: 1.8.1                  __init__.py:112
(trainable_simulations pid=28552) 
(trainable_simulations pid=28552) Elapsed run time = 63.1702 s
(trainable_simulations pid=28552) 
(trainable_simulations pid=28552) 2023-01-12 18:46:51.592 | INFO     | gdsfactory.config:<module>:48 - Load '/workspaces/gdsfactory/gdsfactory' 6.18.3
(trainable_simulations pid=28552) 2023-01-12 18:46:51.594 | INFO     | gdsfactory.config:<module>:48 - Load '/workspaces/gdsfactory/gdsfactory' 6.18.3
(trainable_simulations pid=28552) 2023-01-12 18:46:56.464 | INFO     | gdsfactory.simulation.gmeep:<module>:34 - Meep '1.25.0-beta' installed at ['/opt/conda/lib/python3.8/site-packages/meep']
(trainable_simulations pid=28552) 2023-01-12 18:46:56.465 | INFO     | gdsfactory.simulation.gmeep:<module>:34 - Meep '1.25.0-beta' installed at ['/opt/conda/lib/python3.8/site-packages/me

The results can be seen and manipulated in DataFrame

In [7]:
df = results.get_dataframe()
df

,loss,value,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,...,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,config/length_mmi,config/width_mmi,logdir
0,0.044482,0.255518,69.491283,False,NaN,NaN,1,9aa03cd4,dc732386faaf4f5d8c067b87b7a93768,2023-01-12_18-28-00,...,20689,b5fb30c725ce,172.17.0.2,69.491283,0,1,0.003316,0.619846,1.569637,/workspaces/gdsfactory/tune/ray_results/traina...
1,0.112271,0.187729,67.180003,False,NaN,NaN,1,dbe95eac,c6a6f33c04de44998fed3ead0c972aeb,2023-01-12_18-28-04,...,20796,b5fb30c725ce,172.17.0.2,67.180003,0,1,0.004205,0.166437,1.280155,/workspaces/gdsfactory/tune/ray_results/traina...
2,0.237125,0.062875,59.856010,False,NaN,NaN,1,146c3d00,71ce8e5fcc2e4e00b7d86e24407b0818,2023-01-12_18-29-06,...,21158,b5fb30c725ce,172.17.0.2,59.856010,0,1,0.003072,0.925403,0.352931,/workspaces/gdsfactory/tune/ray_results/traina...
3,0.042641,0.257359,56.970690,False,NaN,NaN,1,c514682f,34524ca57add440f998a35a2ffeb238e,2023-01-12_18-29-11,...,21262,b5fb30c725ce,172.17.0.2,56.970690,0,1,0.003473,0.374672,1.874811,/workspaces/gdsfactory/tune/ray_results/traina...
4,0.091460,0.391460,52.957685,False,NaN,NaN,1,052ffa1e,71843c1534094b8caa1da4a7963e79e5,2023-01-12_18-30-05,...,21600,b5fb30c725ce,172.17.0.2,52.957685,0,1,0.003588,1.611635,1.668814,/workspaces/gdsfactory/tune/ray_results/traina...
5,0.077591,0.377591,52.787365,False,NaN,NaN,1,6e18bad1,5a2236f26b1746189ebfbcf2de56f315,2023-01-12_18-30-11,...,21702,b5fb30c725ce,172.17.0.2,52.787365,0,1,0.003649,1.621135,1.921334,/workspaces/gdsfactory/tune/ray_results/traina...
6,0.171732,0.128268,44.443164,False,NaN,NaN,1,f6090086,c7d7730226414dafa5d71f9dfe833f87,2023-01-12_18-30-56,...,22021,b5fb30c725ce,172.17.0.2,44.443164,0,1,0.002802,1.086679,0.605129,/workspaces/gdsfactory/tune/ray_results/traina...
7,0.184728,0.115272,50.695376,False,NaN,NaN,1,4d80c5c4,472382d103284e87820c421c28ad9e4c,2023-01-12_18-31-08,...,22162,b5fb30c725ce,172.17.0.2,50.695376,0,1,0.004440,1.884988,0.923232,/workspaces/gdsfactory/tune/ray_results/traina...
8,0.283638,0.016362,60.886133,False,NaN,NaN,1,a521061d,c53363862f824f76a9b9417c8e42dbad,2023-01-12_18-32-03,...,22419,b5fb30c725ce,172.17.0.2,60.886133,0,1,0.003191,0.861429,0.751978,/workspaces/gdsfactory/tune/ray_results/traina...
9,0.206149,0.093851,63.225708,False,NaN,NaN,1,63d1a023,0971444754d74d1994f47e95aad84cce,2023-01-12_18-32-18,...,22583,b5fb30c725ce,172.17.0.2,63.225708,0,1,0.005080,1.629117,0.965458,/workspaces/gdsfactory/tune/ray_results/traina...


There are clearly many possible solutions, so making a [Pareto front](https://en.wikipedia.org/wiki/Pareto_front) plot w.r.t. some other parameter like overall size would make sense here.

In [8]:
best_params = results.get_best_result(metric="loss", mode="min").metrics
best_params['loss'], best_params['config']

(0.006401742005686328,
 {'length_mmi': 1.1593657286109003, 'width_mmi': 1.4685922896289638})